Счётчик бактерий в чашке Петри.

1) При подготовке датасета думал как решить проблему большого разрешения и мелких объектов при детекции. Прочитал статью https://blog.roboflow.com/detect-small-objects/

Оттуда понял, как можно эту проблему решить. 

Также в статье есть советы по улучшению качества модели:

    Increasing your image capture resolution
    Increasing your model's input resolution
    Tiling your images
    Generating more data via augmentation
    Auto learning model anchors
    Filtering out extraneous classes 

Это помимо очевидных: попробовать другие архитектуры сетей, дольше обучать и т.д.

2) На робофлоу порезал изображения и аннотации на куски: 5 частей по одной оси, 10 по другой. В итоге получилось больше почти квадратных имеджей меньшего разрешения (1040х911) и аннотации к ним. Изображения без аннотированных объектов убрал, чтобы не засорять датасет. Аугментацию не проводил, чтобы датасет не разроссься и не увеличил время обучения сети.

3) Дальше просто шёл по коду. Причина выбора такой архитектуры в комментарии перед созданием экземпляра сети.

4) Дорофее придётся нарезать снимки, прежде чем отправлять их на инференс.

5) Уверенность у модели низкая, по причинам, указанным выше (не все советы по улучшению были применены), поэтому о точности также говорить не приходится.





Монтируем drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Установка зависимостей, импорт библиотек. Ошибки, warning-и - ничего страшного.

In [ ]:
!apt -y install libportaudio2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 65.4 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Fetched 65.4 kB in 1s (66.8 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 122352 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1build1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1build1) ...
Setting up libportaudio2:amd64 (19.6.0-1build1) ...
Processing triggers for libc-bin (2.31-0ubuntu9.9) ...


In [ ]:
pip install tflite-support

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 34.6 MB/s eta

In [ ]:
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

ERROR: Could not find a version that satisfies the requirement opencv-python-headless==4.1.2.30 (from versions: 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.14.51, 3.4.14.53, 3.4.15.55, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.2.52, 4.5.2.54, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66, 4.7.0.68, 4.7.0.72)
ERROR: No matching distribution found for opencv-python-headless==4.1.2.30
Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 27.9 MB/

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)


/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're usin

Выбрал эту сетку (efficientdet_lite4), потому что впервые наткнулся на неё. Она лёгкая, быстро обучается. 
По уму тут нужно выбирать архитектуру, которая показывает более высокую уверенность классификации. И не смотреть на время инференса - оно не важно, как по мне, для задачи подсчёта бактерий в чашке.


In [ ]:
spec = model_spec.get('efficientdet_lite4')

Распаковка архива, залитого на гугл-драйв.
В архиве нарезанные изображения + xml-аннотации.

In [ ]:
!unzip -q /content/drive/MyDrive/Some_folder/test_2.zip

Подготовка данных перед обучением

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    '/content/train',
    '/content/train',
    ['Dense', 'Diffuse', 'Mixed']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/content/valid',
    '/content/valid',
    ['Dense', 'Diffuse', 'Mixed']
)

Обучение

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=3, train_whole_model=True, epochs=14, validation_data=val_data)

Epoch 1/14
183/183 [==============================] - 214s 781ms/step - det_loss: 1.2712 - cls_loss: 0.8210 - box_loss: 0.0090 - reg_l2_loss: 0.1080 - loss: 1.3792 - learning_rate: 0.0059 - gradient_norm: 4.4651 - val_det_loss: 0.9982 - val_cls_loss: 0.6002 - val_box_loss: 0.0080 - val_reg_l2_loss: 0.1082 - val_loss: 1.1064
Epoch 2/14
183/183 [==============================] - 133s 727ms/step - det_loss: 0.8664 - cls_loss: 0.5351 - box_loss: 0.0066 - reg_l2_loss: 0.1082 - loss: 0.9746 - learning_rate: 0.0036 - gradient_norm: 4.8735 - val_det_loss: 0.8185 - val_cls_loss: 0.4760 - val_box_loss: 0.0068 - val_reg_l2_loss: 0.1082 - val_loss: 0.9267
Epoch 3/14
183/183 [==============================] - 133s 725ms/step - det_loss: 0.8085 - cls_loss: 0.4988 - box_loss: 0.0062 - reg_l2_loss: 0.1083 - loss: 0.9168 - learning_rate: 0.0034 - gradient_norm: 4.7444 - val_det_loss: 0.8873 - val_cls_loss: 0.4910 - val_box_loss: 0.0079 - val_reg_l2_loss: 0.1083 - val_loss: 0.9956
Epoch 4/14
183/183 [==

Экспорт модели.

In [ ]:
model.export(export_dir='/content/drive/MyDrive/Some_folder', tflite_filename='petry.tflite')

Код и импорты для инференса.

In [ ]:
from tflite_support.task import vision
from tflite_support.task import core
from tflite_support.task import processor

In [ ]:
model_path="/content/drive/MyDrive/Some_folder/petry.tflite"
detector = vision.ObjectDetector.create_from_file(model_path)

In [ ]:
img_path = "/content/train/-10_jpg.rf.105d5667d8746dda75ac670aa422c3de.jpg"
image = vision.TensorImage.create_from_file(img_path)
detection_result = detector.detect(image)

Просто счётчик всех классов по отдельности и вместе.

In [ ]:
import re

flora_sum = 0
for i in ['Dense', 'Diffuse', 'Mixed']:
  counter = len(re.findall(f"{i}", str(detection_result)))
  print(i, counter)
  flora_sum += counter

print(f'Total: {flora_sum}')

Dense 11
Diffuse 0
Mixed 14
Total: 25


END